In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
from sklearn.mixture import GaussianMixture
from tensorflow.keras.regularizers import l2

# Load datasets and initialize models
for n in [30]:
    # Load dataset
    import os
    print("Current Directory:", os.getcwd())
    Xs = np.load(f"/home/moritz/maths-for-ml/Kryptonite-N/Datasets/kryptonite-{n}-X.npy")
    Ys = np.load(f"/home/moritz/maths-for-ml/Kryptonite-N/Datasets/kryptonite-{n}-y.npy")
    df_x = pd.DataFrame(Xs)
    df_y = pd.Series(Ys)  # Use Series if Ys is 1D
    # Apply Gaussian Mixture Model to capture bimodality
    # ... existing code ...

    # Apply Gaussian Mixture Model to each feature individually
    df_x_transformed = pd.DataFrame()

    for column in df_x.columns:
        gmm = GaussianMixture(n_components=2, random_state=42)
        gmm.fit(df_x[[column]])
        proba = gmm.predict_proba(df_x[[column]])
        df_x_transformed[f'{column}_Mode_Prob'] = np.where(proba[:, 0] > proba[:, 1], -proba[:, 0], proba[:, 1])

    df_x_combined = pd.concat([(df_x >= 0.5).astype(int), df_x_transformed], axis=1)

    print(df_x_combined.shape)
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(df_x_combined, df_y, test_size=0.2, random_state=42)

    # Build the neural network model
   

    model = Sequential([
    Input(shape=(X_train.shape[1],)),

    Dense(256),
    LeakyReLU(),
    BatchNormalization(),
    
    Dense(128),
    LeakyReLU(),
    BatchNormalization(),

    Dense(64),
    LeakyReLU(),
    BatchNormalization(),

    Dense(1, activation='sigmoid')
])


    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    # Define a callback to stop training when accuracy reaches 95%
    class EarlyStoppingByAccuracy(Callback):
        def on_epoch_end(self, epoch, logs=None):
            if logs.get('accuracy') >= 0.94:
                print("\nReached 94% accuracy, stopping training!")
                self.model.stop_training = True

    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=150,
        batch_size=32,
        validation_split=0.1, 
        verbose=1,
        callbacks=[EarlyStoppingByAccuracy()]
    )

    # Make predictions and evaluate
    y_pred_nn = (model.predict(X_test) > 0.5).astype(int)
    accuracy_nn = accuracy_score(y_test, y_pred_nn)
    print(f"Accuracy of the Neural Network Classifier on test set for n = {n}: {accuracy_nn:.4f}")
    print(classification_report(y_test, y_pred_nn))


Current Directory: \\wsl.localhost\Ubuntu\home\moritz\maths-for-ml
(60000, 60)
Epoch 1/150
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.4911 - loss: 0.7401 - val_accuracy: 0.4981 - val_loss: 0.6988
Epoch 2/150
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5137 - loss: 0.6966 - val_accuracy: 0.4902 - val_loss: 0.6991
Epoch 3/150
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5171 - loss: 0.6943 - val_accuracy: 0.4981 - val_loss: 0.6961
Epoch 4/150
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5218 - loss: 0.6933 - val_accuracy: 0.5167 - val_loss: 0.6962
Epoch 5/150
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5207 - loss: 0.6928 - val_accuracy: 0.5035 - val_loss: 0.6982
Epoch 6/150
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5208 - loss: 0.6928 - val_accuracy: 0.4992 - val_loss: 0.6957
Epoch 7/150
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5251 - loss: 0.6916 - val_accuracy: 0.4981 - val_loss: 0.6960
Epoc

KeyboardInterrupt: 

In [13]:
# Save the model as moritz-n24
model.save('moritz-n24.keras')
